In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")

In [42]:
#mylist = []

#for chunk in pd.read_csv('C://duh/Thinkful/LoanStats3d.csv', chunksize=2000):
#    mylist.append(chunk)

#y2015 = pd.concat(mylist, axis= 0)
#del mylist
# Replace the path with the correct path for your data.
y2015 = pd.read_csv('C://duh/Thinkful/LoanStats3d.csv', engine='python')
#y2015.info(memory_usage='deep')

In [43]:
# Convert ID and Interest Rate to numeric.
y2015['term'] = pd.to_numeric(y2015['term'].str.strip('months'), errors='coerce')
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')
# Drop other columns with many unique variables
y2015.drop(['total_il_high_credit_limit',
            'total_bc_limit',
            'total_bal_ex_mort',
            'tot_hi_cred_lim',
            'num_sats',
            'num_rev_tl_bal_gt_0',
            'num_rev_accts',
            'num_op_rev_tl',
            'num_il_tl',
            'num_bc_tl',
            'num_bc_sats',
            'num_actv_rev_tl',
            'num_actv_bc_tl',
            'mort_acc',
            'bc_util',
            'bc_open_to_buy',
            'avg_cur_bal',
            'acc_open_past_24mths',
            'inq_last_12m',
            'total_rev_hi_lim',
            'open_rv_24m',
            'open_rv_12m',
            'total_bal_il',
            'open_act_il',
            'open_acc_6m',
            #'tot_cur_bal',
            'last_pymnt_amnt',
            #'collection_recovery_fee',
            #'recoveries',
            'total_rec_late_fee',
            'total_rec_int',
            'total_rec_prncp',
            'total_pymnt_inv',
            'total_pymnt',
            'out_prncp_inv',
            'out_prncp',
            #'total_acc',
            #'revol_bal',
            #'open_acc',
            #'annual_inc',
            #'installment',
            #'int_rate',
            #'term',
            #'funded_amnt_inv',
            #'funded_amnt',
            #'loan_amnt'
           ], 1, inplace=True)

## DRILL: Third Attempt

So here's your task. Get rid of as much data as possible without dropping below an average of 90% accuracy in a 10-fold cross validation.

You'll want to do a few things in this process. First, dive into the data that we have and see which features are most important. This can be the raw features or the generated dummies. You may want to use PCA or correlation matrices.

Can you do it without using anything related to payment amount or outstanding principal? How do you know?

In [44]:
y2015.corr()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,annual_inc,open_acc,revol_bal,total_acc,recoveries,collection_recovery_fee,tot_cur_bal
loan_amnt,1.000000,1.000000,0.999994,0.406855,0.140496,0.941166,0.305734,0.188214,0.334656,0.208430,0.148936,0.148728,0.325352
funded_amnt,1.000000,1.000000,0.999994,0.406855,0.140496,0.941166,0.305734,0.188214,0.334656,0.208430,0.148936,0.148728,0.325352
funded_amnt_inv,0.999994,0.999994,1.000000,0.406743,0.140133,0.941148,0.305803,0.188212,0.334664,0.208464,0.148860,0.148649,0.325401
term,0.406855,0.406855,0.406743,1.000000,0.438650,0.127826,0.047360,0.086865,0.086874,0.101139,0.127442,0.126600,0.102172
int_rate,0.140496,0.140496,0.140133,0.438650,1.000000,0.124431,-0.090357,-0.012968,-0.057468,-0.040191,0.183950,0.184651,-0.086973
installment,0.941166,0.941166,0.941148,0.127826,0.124431,1.000000,0.297394,0.169987,0.314070,0.183499,0.141862,0.142075,0.295472
annual_inc,0.305734,0.305734,0.305803,0.047360,-0.090357,0.297394,1.000000,0.122328,0.270174,0.159804,0.020681,0.020166,0.379008
open_acc,0.188214,0.188214,0.188212,0.086865,-0.012968,0.169987,0.122328,1.000000,0.222987,0.709499,0.041277,0.041444,0.245249
revol_bal,0.334656,0.334656,0.334664,0.086874,-0.057468,0.314070,0.270174,0.222987,1.000000,0.180191,0.018584,0.018458,0.455735
total_acc,0.208430,0.208430,0.208464,0.101139,-0.040191,0.183499,0.159804,0.709499,0.180191,1.000000,0.045894,0.046183,0.305456


In [45]:
# Your code here.
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

cross_val_score(rfc, X, Y, cv=10)

c:\users\lolium inter spinas\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\model_selection\_split.py:652: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
c:\users\lolium inter spinas\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\lolium inter spinas\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\lolium inter spinas\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\ensemble\forest.py:246: 

array([0.92239926, 0.92676672, 0.9262663 , 0.92700071, 0.92894799,
       0.93257973, 0.93293595, 0.93001164, 0.93058161, 0.9345477 ])